<a href="https://colab.research.google.com/github/tiyu0203/fml/blob/master/fml_proj7_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 475 Project 7: Recommender Systems (& NMF)
Tiffany Yu, Jonathan Lam, Harris Paspuleti

> To implement a basic recommendation system. Many of those datasets are already loaded into the Surpise package to make this easy. You should tune and cross validate your system to select the best values for the # of latent dimensions, the regularization parameter, and any other hyperparameters.

In [ ]:
# get surprise package for NMF
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 8.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670943 sha256=804d6c6fafdc481a633606b7f462c7a284a9177d87f39128828be1f9e86e1a11
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection.search import GridSearchCV
from surprise.model_selection.split import train_test_split
from surprise import accuracy

## Dataset: MovieLens 100K

> MovieLens 100K movie ratings. Stable benchmark dataset. 100,000 ratings from 1000 users on 1700 movies. Released 4/1998.

This dataset includes a set of user details, movie details, and user ratings for movies. Each user rated at least 20 movies.

For our current analysis, we do not use the user nor the movie details (only use the ratings information).

In [ ]:
# Load movielens-100k dataset (https://surprise.readthedocs.io/en/stable/dataset.html)
data = Dataset.load_builtin('ml-100k')

# Split into training and testing sets
train, test = train_test_split(data, test_size=0.2)

# show the first few ratings: tuples of (user, movie, rating)
some_ratings = []
for i, rating in enumerate(train.all_ratings()):
  if i > 20:
    break;
  some_ratings.append(rating)
  
print("Here is a sample of the ratings to show their format:")
some_ratings

Here is a sample of the ratings to show their format:


[(0, 0, 3.0),
 (0, 425, 5.0),
 (0, 421, 5.0),
 (0, 565, 3.0),
 (0, 396, 5.0),
 (0, 698, 4.0),
 (0, 867, 4.0),
 (0, 27, 2.0),
 (0, 888, 3.0),
 (0, 371, 2.0),
 (0, 253, 3.0),
 (0, 753, 5.0),
 (0, 904, 4.0),
 (0, 1009, 3.0),
 (0, 62, 2.0),
 (0, 1013, 2.0),
 (0, 49, 3.0),
 (0, 166, 4.0),
 (0, 820, 3.0),
 (0, 342, 3.0),
 (0, 707, 2.0)]

## Hyperparameter tuning

We use the Surprise libary's builtin `GridSearchCV` for automatic hyperparameter tuning and cross-validation.

All models learn a bias for the user and movie matrices, and use the default number of epochs (50).

Explanation of hyperparameters:
- `n_factors`: Dimensionality of the latent space
- `reg_pu`: Regularization coefficient for the $p_u$ (user) matrix
- `reg_qi`: Regularization coefficient for the $q_i$ (item/movie) matrix
- `reg_bu`: Regularization coefficient for the user matrix bias
- `reg_bi`: Regularization coefficient for the movie matrix bias

For sake of time, we did not try very many hyperparameter combinations.

In [ ]:
# grid search 
params = {
    'n_factors': [10, 20, 30, 40],
    'reg_pu': [0.02, 0.002],
    'reg_qi': [0.02, 0.002],
    'biased': [True],
    'reg_bu': [0.02, 0.002],
    'reg_bi': [0.02, 0.002]
}
grid_search = GridSearchCV(NMF, params, refit=True,
                           return_train_measures=True, n_jobs=-1)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']

Just to get an idea of what our model looks like (and make sure it is learning), print out some of the learned parameters and the cross validation results:

In [ ]:
print(grid_search.best_score, grid_search.best_params, grid_search.cv_results)

{'rmse': 0.953552256033319, 'mae': 0.7469554124946353} {'rmse': {'n_factors': 10, 'reg_pu': 0.02, 'reg_qi': 0.02, 'biased': True, 'reg_bu': 0.002, 'reg_bi': 0.002}, 'mae': {'n_factors': 10, 'reg_pu': 0.02, 'reg_qi': 0.02, 'biased': True, 'reg_bu': 0.002, 'reg_bi': 0.002}} {'split0_test_rmse': array([1.12671965, 0.95368045, 0.95090741, 0.95168667, 0.96127206,
       0.96771622, 0.96343015, 0.97002468, 1.10883185, 0.96771741,
       0.9631692 , 0.96989675, 0.97158942, 0.9766609 , 0.97787778,
       0.98198263, 1.31584452, 1.18207468, 1.2660726 , 1.23342571,
       1.27359044, 1.31343888, 1.45571804, 1.47059832, 1.47713136,
       1.90276703, 1.30630309, 1.25483712, 1.80809646, 1.76598118,
       1.52439289, 1.60655147, 1.38715191, 1.2228146 , 1.28089097,
       1.46255891, 1.93239573, 1.68333947, 1.44744428, 2.32940068,
       1.31393296, 1.39052051, 1.6002692 , 1.29269805, 1.61572357,
       1.36836678, 1.71614943, 1.7443026 , 1.51647067, 1.48985898,
       1.23764595, 1.25609113, 1.323

## Evaluating the model
In this section, we evaluate the model on all of the test dataset ratings, and manually calculate the RMSE and accuracy.

In [ ]:
# Fit and test the model
predictions = algo.test(test)

prediction_array = []
for prediction in predictions:
  uid = int(prediction.uid)
  iid = int(prediction.iid)
  r_ui = int(prediction.r_ui)
  est = float(prediction.est)
  prediction_array.append([uid, iid, r_ui, est])

pred = np.array(prediction_array)

# calculating RMSE
rmse = np.sqrt(np.mean((pred[:,2] - pred[:,3]) ** 2))

# calculating accuracy
est_rounded = pred[:,3].astype(int)
accuracy = np.sum(pred[:,2] != est_rounded) / pred.shape[0]

# print out test metrics
print(f'RMSE: {rmse}; Accuracy: {accuracy}')

# print out some predictions
print('Test dataset ratings vs. model predictions:')
print(np.vstack((pred[:,2], pred[:,3])).T)

RMSE: 0.8450194768270142; Accuracy: 0.6539
Test dataset ratings vs. model predictions:
[[5.         3.79930586]
 [5.         3.58003842]
 [3.         3.28499177]
 ...
 [3.         3.28001867]
 [4.         3.87734224]
 [4.         2.30047792]]


## Making recommendations for a user

In [ ]:
# do predictions!
num_users = len(train.all_users())
num_items = len(train.all_items())

# read in user and movie data
# (this was not used when making the recommender systems, but it's here so we can get some context)
user_data = pd.read_csv('https://raw.githubusercontent.com/tiyu0203/fml/master/u.user', '|', header=None)
item_data = pd.read_csv('https://raw.githubusercontent.com/tiyu0203/fml/master/u.item', '|', header=None)

# choose some arbitrary user
uid = 152

# get all of the first user's rated movies
all_ratings = []
for rating in train.all_ratings():
  all_ratings.append([*rating])
all_ratings = np.array(all_ratings)

# get all ids of movies that the user rates
user_rated_movies = all_ratings[all_ratings[:,0] == uid,:]
user_rated_movie_titles = item_data.loc[user_rated_movies[:,1], 1]
user_rated_movie_ratings = user_rated_movies[:,2]
print('Rated movies: ', np.vstack((user_rated_movie_titles, user_rated_movie_ratings)).T)

# predict highest recommendations (estimated >4.9 for this particular user)
recommendations = []
for i in range(num_items):
  if algo.predict(str(uid), str(i)).est > 4.9:
    recommendations.append(i)
recommendations = np.array(recommendations)
print('Recommended movies: \n', item_data.loc[recommendations, 1])

# show user information as well
print('User: ', user_data.loc[uid,:])

Rated movies:  [['Ace Ventura: Pet Detective (1994)' 3.0]
 ['Tales From the Crypt Presents: Demon Knight (1995)' 5.0]
 ['Patton (1970)' 3.0]
 ['Jurassic Park (1993)' 3.0]
 ['Juror, The (1996)' 4.0]
 ['Dragonheart (1996)' 3.0]
 ["Preacher's Wife, The (1996)" 3.0]
 ['Madness of King George, The (1994)' 4.0]
 ['Nadja (1994)' 5.0]
 ['Little Women (1994)' 5.0]
 ['To Wong Foo, Thanks for Everything! Julie Newmar (1995)' 4.0]
 ['Trainspotting (1996)' 5.0]
 ['Pulp Fiction (1994)' 4.0]
 ['Tales from the Hood (1995)' 4.0]
 ['Four Days in September (1997)' 3.0]
 ['Thin Man, The (1934)' 3.0]
 ['Jungle2Jungle (1997)' 2.0]
 ['Independence Day (ID4) (1996)' 1.0]
 ['Crow, The (1994)' 4.0]
 ['From Dusk Till Dawn (1996)' 4.0]
 ["Jackie Chan's First Strike (1996)" 5.0]
 ['Sting, The (1973)' 3.0]
 ['Sleepless in Seattle (1993)' 2.0]
 ['Last Dance (1996)' 2.0]
 ['Conan the Barbarian (1981)' 3.0]
 ['My Life as a Dog (Mitt liv som hund) (1985)' 3.0]
 ['Henry V (1989)' 3.0]
 ['Sudden Death (1995)' 4.0]
 ['Bas

---

# Stretch goal #1
> Implement non-negative matrix factorization (NMF) using alternating least squares (ALS)

I.e., solve:

$$\min_{q,p}\sum_{(u,i)\in\kappa}(r_{ui}-q_i^Tp_u)^2+\lambda(||q_i||^2+||p_u||^2)$$

(assuming $q_i$ and $p_u$ are column vectors)

Do this by minimizing $q_i^T$ while keeping $p_u$ fixed and vice versa and repeating until convergence.

See: https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea

As a matrix problem (ridge regression):

$$\min_{P_u,Q_i}(R_{ui}-P_uQ_i^T)^T(R_{ui}-P_uQ_i^T)+\lambda_u||P_u||^2+\lambda_i||Q_i||^2$$

Each row of $Q_i$ and $P_u$ is a item/user. (Let $f$ denote latent space dimension, $i$ denote the number of items, and $u$ denote the number of users.)

$$P_u\in M_{u\times f}$$
$$Q_i\in M_{i\times f}$$
$$R_{ui}\in M_{u\times i}$$

Update rules:

$$p_u^T\leftarrow r_u^TQ_i(Q_i^TQ_i+\lambda_uI_f)^{-1}$$
$$q_i^T\leftarrow r_i^TP_u(P_u^TP_u+\lambda_uI_f)^{-1}$$

(Here $r_u$ and $r_i$ are also column vectors, i.e., $r_u$ is the transpose of the $u$th row of $R_{ui}$, and $r_i$ is the $i$th row of $R_{ui}$.)

## Model

This performs ALS as an algorithm for NMF. We did not implement a learned bias (this is similar to using `biased=False` in the Surprise library).

---

#### `__init__`
Initializes the model. Generates the (sparse) $R_{ui}$ matrix, and initializes $P_u$ and $Q_i$ using random normal distributions. 

Parameters:
- `dataset`: training dataset; tuples of `(user, item, rating)` in the format of the Surprise training dataset
- `n_factors`: latent dimension (default 15)
- `lambda_u`: L2 regularization coefficient for `P_u` (default 0.02)
- `lambda_i`: L2 regularization coefficient for `Q_i` (default 0.02)

---

#### `loss`
Calculates the loss on the training dataset. Used during training.

---

#### `update`
Performs the update rule on $P_u$ (once for every user) and $Q_i$ (once for every item). (This is considered one epoch).

By fixing $Q_i$, we can solve optimally for $P_u$ (since this is a (quadratic) ridge regression problem), and vice versa. However, $R_{ui}$ is sparse and we only want to train on the rated items (otherwise we would be training users to give most movies a zero rating). To accomplish this, we can only train on a single user at a time, only using the vector of movies that the user rates and "filtering" the items matrix $Q_i$ to those same items. The same applies when solving for $Q_i$.

---

#### `train`
Performs the update rule `epoch` times, and reports the training loss.

Parameters:
- `epochs`: number of epochs to train

---

#### `predict`
Predict the rating for a given user and item.

Parameters:
- `user`: (integral) id of user to predict
- `item`:  (integral) id of item to predict

---

#### `predict_all`
Predict the ratings for every user for every item (i.e., estimate the full, dense $R_{ui}$).

---

#### `test`
Predict the ratings given user, item pairs.

Parameters:
- `dataset`: Test dataset, tuples of `(user, item, rating)`

In [ ]:
class ALS_NMF():

    # assumes dataset in the same format as the one given by the surprise library
    def __init__(self, dataset, n_factors=15, lambda_u=0.02, lambda_i=0.02):
        self.user_count, self.item_count = 0, 0
        for _ in dataset.all_users(): self.user_count += 1
        for _ in dataset.all_items(): self.item_count += 1

        self.R_ui = np.zeros((self.user_count, self.item_count))

        self.ratings = []
        for rating in dataset.all_ratings():
            self.ratings.append(rating)
            self.R_ui[rating[0], rating[1]] = rating[2]

        self.ratings = np.array(self.ratings, dtype=np.int)
        self.N = self.ratings.shape[0]

        # initializes P_u and Q_i
        self.P_u = np.random.normal(size=(self.user_count, n_factors))
        self.Q_i = np.random.normal(size=(self.item_count, n_factors))

        self.n_factors = n_factors

        self.lambda_u = lambda_u
        self.lambda_i = lambda_i
  
    # calculate loss (to check if update rule works)
    def loss(self):
        losses = np.zeros(self.N)
        for j, rating in enumerate(self.ratings):
            u, i, r = rating
            u, i = int(u), int(i)
            losses[j] = (r - self.P_u[u, :] @ self.Q_i[i, :].T) ** 2

        return np.mean(losses) + self.lambda_u*np.linalg.norm(self.P_u) + self.lambda_i*np.linalg.norm(self.Q_i) 
    
    # update rule
    def update(self):
        # assume Q_i is fixed, update P_u
        # loop through each user
        for u in range(self.user_count):
            # find the items that this user has rated
            unfiltered_user_ratings = self.R_ui[u, :]
            user_rated_items = unfiltered_user_ratings > 0

            # "filtered" things
            r_u = unfiltered_user_ratings[user_rated_items]

            Q_i = self.Q_i[user_rated_items, :]

            # convex quadratic optimal soln
            # p_u^T = r_u @ Q_i @ (Q_i^T @ Q_i + lambda_u * I_f)^-1
            self.P_u[u, :] = r_u @ Q_i @ np.linalg.pinv(Q_i.T @ Q_i + self.lambda_u * np.eye(self.n_factors))

        # assume P_u is fixed, update Q_i
        # loop through each item
        for i in range(self.item_count):
            # find the users that rated this item
            unfiltered_item_ratings = self.R_ui[:, i].T
            users_rating_this_item = unfiltered_item_ratings > 0

            # "filtered" things
            r_i = unfiltered_item_ratings[users_rating_this_item]
            P_u = self.P_u[users_rating_this_item, :]

            # convex quadratic optimal soln
            self.Q_i[i, :] = r_i @ P_u @ np.linalg.inv(P_u.T @ P_u + self.lambda_i * np.eye(self.n_factors))

    # train method
    def train(self, epochs):
        print(f'Initial loss: {self.loss()}')
        for i in range(epochs):
            self.update()
            # for j, rating in enumerate(self.ratings):
            #     self.update(rating)
            print(f'Epoch: {i}; Loss: {self.loss()}')

    # evaluate
    # r_ui = p_u^T @ q_i
    def predict(self, user, item):
        if user.item >= self.user_count or i >= self.item_count:
            est = np.random.random() * 5
        else:
            est = self.P_u[user, :] @ self.Q_i[item, :].T
        return est

    # predict cross product of all ratings for training set
    def predict_all(self):
        return self.P_u @ self.Q_i.T

    # this assumes that the test dataset is a list of tuples (unlike the train dataset)
    def test(self, dataset):
        ratings = []
        for u, i, r in (dataset if isinstance(dataset, list) else dataset.all_ratings()):
            ratings.append([self.predict(int(u), int(i)), r])
        return np.array(ratings)

##Choose the parameters and train the model

When choosing the lambdas, if we chose a higher number the loss would converge to a much larger number; if we chose a small number, the loss would converge to a really small number as well as decrease in a steeper exponential decay. We left `n_factors` the same as the default from the NMF function in the surprise library. After testing a variety of numbers for epochs, we realized that the loss converged relatively quickly so we wouldn't need many epochs. 

In [ ]:
# choose regularization coefficient
lambda_u = 0.02
lambda_i = 0.02

# choose f
n_factors = 15

# choose epochs
epochs = 50

# run
model = ALS_NMF(train, n_factors, lambda_u, lambda_i)
model.train(epochs)

#Regularization coefficient for the p_u (user) matrix
P_u = model.P_u()
#Regularization coefficient for the q_i (item/movie) matrix
Q_i = model.Q_i()

print(f'P_u: {P_u}, Q_i: {Q_i}')

Initial loss: 34.80673227663466
Epoch: 0; Loss: 11.626744968095991
Epoch: 1; Loss: 7.1488177789313525
Epoch: 2; Loss: 6.855198026518326
Epoch: 3; Loss: 6.710935612941744
Epoch: 4; Loss: 6.60877224014469
Epoch: 5; Loss: 6.537936217060015
Epoch: 6; Loss: 6.484704692789716
Epoch: 7; Loss: 6.441820974992813
Epoch: 8; Loss: 6.40476382115052
Epoch: 9; Loss: 6.371439849812138
Epoch: 10; Loss: 6.343120364076325
Epoch: 11; Loss: 6.317881711400218
Epoch: 12; Loss: 6.293993108763518
Epoch: 13; Loss: 6.270658686126371
Epoch: 14; Loss: 6.249897929032098
Epoch: 15; Loss: 6.230931581722405
Epoch: 16; Loss: 6.215299061388394
Epoch: 17; Loss: 6.20149670735049
Epoch: 18; Loss: 6.189626131529177
Epoch: 19; Loss: 6.1799425362648055
Epoch: 20; Loss: 6.172100212112769
Epoch: 21; Loss: 6.165451455079193
Epoch: 22; Loss: 6.159029255519029
Epoch: 23; Loss: 6.153080408069977
Epoch: 24; Loss: 6.147599787424625
Epoch: 25; Loss: 6.142599610421463
Epoch: 26; Loss: 6.137694837852222
Epoch: 27; Loss: 6.13285558604546

## Calculating the RMSE and Accuracy for Test dataset
Our RMSE on the test dataset is low, but the accuracy is no better than random guessing. We were not able to diagnose why this is the case. Increasing the regularization parameters to try to prevent overfitting did not help. In the next section we also test on the training dataset and show that does train well, so we are not sure why it does not generalize past the training dataset.

In [ ]:
test_res = model.test(test)
rmse = np.sqrt(np.mean((test_res[:, 0] - test_res[:, 1]) ** 2))
accuracy = np.mean(np.round(test_res[:, 0]) == np.round(test_res[:, 1]))
print(f'RMSE: {rmse}, Accuracy: {accuracy}')

RMSE: 1.9356349226785399, Accuracy: 0.21745


## Calculating the RMSE and Accuracy for the Train dataset

As a sanity check, we ran the train dataset in our model to see if there was an issue on how we implemented the algorithm. This indicates that our model does indeed train correctly, so we are not sure why it doesn't generalize.

In [ ]:
test_res = model.test(train)
rmse = np.sqrt(np.mean((test_res[:, 0] - test_res[:, 1]) ** 2))
accuracy = np.mean(np.round(test_res[:, 0]) == np.round(test_res[:, 1]))
print(f'RMSE: {rmse}, Accuracy: {accuracy}')

RMSE: 0.5474335410733868, Accuracy: 0.6905625
